In [1]:
shape_cat="mcal"
vol="vlim"
z_range="all_z"


lambda_range=0
with_r=True

In [2]:
# matplotlib.rcParams.update({'font.size': 11}) # before plotting coode
# plt.figure(figsize=(6.2,5))
# ...
# plt.tight_layout()
# plt.subplots_adjust(hspace=0,wspace=0) # if doing multiple panels/subplots
# plt.savefig('namee.pdf', bbox_inches='tight')

In [3]:
from astropy.io import fits
from astropy.table import Table
import pandas as pd
import fitsio as fio
import numpy as np
import ast

import astropy.cosmology
from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude  # Angles
import astropy.units as u
from IPython.display import display, Math
from tqdm.notebook import tqdm
tqdm.pandas()

/global/homes/z/zchusre/.conda/envs/desdesi/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
#This header file contains all the common packages
%matplotlib inline
%run -i /global/project/projectdirs/des/zhou/sa/lib/src/header.py $shape_cat $vol $z_range 
%run -i /global/project/projectdirs/des/zhou/sa/lib/src/func.py  
%run -i /global/project/projectdirs/des/zhou/sa/lib/src/corr_func.py

members= pd.read_pickle("/global/project/projectdirs/des/zhou/sa/data/BPZ.pkl") #load BPZ catalog
red_members=pd.read_pickle("/global/project/projectdirs/des/zhou/sa/data/shape_BPZ.pkl")
red_members_index=red_members.index
red_members=None

In [5]:
print("The number of total sources is {}".format(len(members)))
print("The number of red sources is {}".format(len(red_members_index)))

The number of total sources is 34714036
The number of red sources is 105029


In [6]:
NBINS=10

z_bins=np.arange(0.2,0.67,0.15)
lambda_bins=np.append(np.arange(20,66,15),1000)

print(z_bins)
print(lambda_bins)

[0.2  0.35 0.5  0.65]
[  20   35   50   65 1000]


In [7]:
woRedMask=~np.isin(members.index,red_members_index)

In [8]:
print("The number of sources excluding redMaPPer is {}".format(np.sum(woRedMask)))
print("The number of red sources being cut is {}".format(np.sum(~woRedMask)))

The number of sources excluding redMaPPer is 34609603
The number of red sources being cut is 104433


In [25]:
center_id=clusters[('Alt','Alt1','ID_CENT')]

In [29]:
members

Band             All                                                         \
Data              e1        e2 flags_select          RA       DEC    mean_z   
ID                                                                            
3007825738 -0.067457 -0.108823            0  318.189343  0.373614  0.592499   
3007825739 -0.241395 -0.168924            0  317.966101  0.373698  1.144830   
3007825742 -0.003933  0.491628            0  317.586931  0.373745  0.493385   
3007825768 -0.005586 -0.118534            0  318.013978  0.374081  1.024960   
3007825777 -0.019404  0.043441            0  317.534582  0.374336  0.417675   
...              ...       ...          ...         ...       ...       ...   
3172249251 -0.116175 -0.202291            0  150.342516  2.505051  1.049819   
3172249272  0.016469 -0.152172            0  149.846985  2.502264  0.694110   
3172249301 -0.129176 -0.203288            0  149.940544  2.498567  0.619950   
3172249302 -0.054150 -0.058570            0  150.311284  2.511910  0.559759   
3172249303  0.222809 -0.212190            0  149.872793  2.512517  0.537623   

Band                     
Data               angR  
ID                       
3007825738   959.643276  
3007825739  1188.569227  
3007825742   875.095067  
3007825768  1162.923418  
3007825777   796.230941  
...                 ...  
3172249251  1169.027836  
3172249272  1028.266779  
3172249301   979.825672  
3172249302   933.825849  
3172249303   915.227695  

[34714036 rows x 7 columns]

In [31]:
np.all(np.isnan(members.reindex([center_id])[('All','RA')]))

True

In [20]:
def get_ng_source(clusters,sources,z_lower,z_upper,lambda_lower,lambda_upper,foreback,woRed=False):
    clusters_z_mask=(clusters[('All','All','Z_LAMBDA')]>=z_lower)&(clusters[('All','All','Z_LAMBDA')]<=z_upper)
    clusters_lambda_mask=(clusters[('All','All','LAMBDA_CHISQ')]>=lambda_lower)&(clusters[('All','All','LAMBDA_CHISQ')]<=lambda_upper)
    
    clusters_masked=clusters[(clusters_z_mask)&(clusters_lambda_mask)]
    
    if woRed==True:
        sources=sources[woRedMask]
    
    drop_poor_centers(clusters_masked)
    
    center_id=clusters_masked[('Alt','Alt1','ID_CENT')]
    
    
    centers=members.reindex([center_id])
    
    
    
    print(np.any(np.isnan(centers[('All','RA')])))
    
    print("The number of clusters is {}".format(len(clusters_masked)))
    print("The number of sources is {}".format(len(sources)))
    
    
    UPPER_BOUND=10
    distance=('All','angR')


#     if mode1 == "s":
#         def get_cluster_cen(cluster):
#             return (get_unique_center_for_cluster(cluster))
#         def get_cluster_sats(cluster):
#             cen_mat_id=cluster.name
#             cen_id=cluster[('Alt','Alt1','ID_CENT')]
            

            
#             if source==True:
#                 sats=members
                
#             else: 
#                 sats=members[members['All','MEM_MATCH_ID']==cen_mat_id]
                         
#             sats=sats[sats.index!=cen_id]
#             return (sats)
#     elif mode1=='r':
#         def get_cluster_sats(cluster):
#             return(shapes)
#         def get_cluster_cen(cluster):
#             return (random)

#     else:
#         raise


    cen=centers
    sats=sources
    
#     if foreback=="back":
#         sats=sats[(sats[('All','mean_z')]-z_upper)>=0.1]
#         print("The number of background sources is {}".format(len(sats)))
#     elif foreback=="fore":
#         sats=sats[(z_lower-sats[('All','mean_z')])>=0.1]
#         print("Calculating foreground sources")
#         print(sats[('All','mean_z')].mean())
#         print("The number of foreground sources is {}".format(len(sats)))

    sats_e1=sats[('All','e1')].to_numpy()
    sats_e2=sats[('All','e2')].to_numpy()
    sats_angr=sats[distance]

#     print(sats_e1,sats_e2)

    if type(cen[('All','RA')])==np.float64:
        cen_angr=np.array(cen[distance]).reshape(1)
        cen_ra=np.array(cen[('All','RA')]).reshape(1)
        cen_dec=np.array(cen[('All','DEC')]).reshape(1)
    else:
#         cen_e1=cen[('All','e1')].to_numpy
#         cen_e2=cen[('All','e2')].to_numpy()
        cen_angr=cen[distance].to_numpy()
        cen_ra=cen[('All','RA')].to_numpy()
        print(cen_ra)
        cen_dec=cen[('All','DEC')].to_numpy()




#     print(cen_e1,cen_e2)

    sats_ra=sats[('All','RA')].to_numpy()
    sats_dec=sats[('All','DEC')].to_numpy()




#     print(np.shape(cen_ra),np.shape(cen_dec))    
#     print(cen_ra,cen_dec)    

    sats_cat = treecorr.Catalog( g1 = sats_e1, g2   = sats_e2, 
                                 ra = sats_ra, dec = sats_dec,r=sats_angr,
                                 ra_units='deg', dec_units='deg')

    cen_cat = treecorr.Catalog( 
                                 ra = cen_ra, dec = cen_dec, r=cen_angr,
                                 ra_units='deg', dec_units='deg')
    
    MINSEP=0.1
    BINSLOP=0.1
        

    ng = treecorr.NGCorrelation(nbins=NBINS, min_sep=MINSEP, max_sep=UPPER_BOUND, bin_slop=BINSLOP,
                               metric="Rperp")
    ng.process(cen_cat,sats_cat)

    return(ng)

In [21]:
## lambda x z = 4x3
import gc

# def get_ng_values(clusters,members,z_bins_low,z_bins_high,lambda_bins_low,lambda_bins_high,foreback):
#     ng=get_ng_source(clusters,members,z_bins_low,z_bins_high,lambda_bins_low,lambda_bins_high,foreback)
#     ng_values=np.copy(np.array([ng.meanlogr,ng.xi,ng.varxi]))
    
#     del ng
    
#     return(ng_values)
    
ng_matrix=[[None for i in range(4)] for j in range(3)]

for i in trange(3):
    for j in trange(4):
        ng=get_ng_source(clusters,members,z_bins[i],z_bins[i+1],lambda_bins[j],lambda_bins[j+1],foreback="")
        ng_values=np.copy(np.array([ng.meanlogr,ng.xi,ng.varxi]))
        ng_matrix[i][j]=ng_values
        del ng
        gc.collect()

The number of clusters before masking 371
The number of clusters with no center shape data 0
The number of clusters after masking 371
True
The number of clusters is 371
The number of sources is 34714036
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan na

ValueError: Catalog has invalid sumw == 0

In [ ]:
np.save("/global/project/projectdirs/des/zhou/sa/data/gammaT/ng_matrix.npy",ng_matrix)
ng_matrix=None

In [ ]:
## lambda x z = 4x3
ng_wr_matrix=[[None for i in range(4)] for j in range(3)]

for i in trange(3):
    for j in trange(4):
        ng=get_ng_source(clusters,members,z_bins[i],z_bins[i+1],lambda_bins[j],lambda_bins[j+1],woRed=True,foreback="")
        ng_values=np.copy(np.array([ng.meanlogr,ng.xi,ng.varxi]))
        ng_wr_matrix[i][j]=ng_values
        del ng
        gc.collect

In [ ]:
np.save("/global/project/projectdirs/des/zhou/sa/data/gammaT/ng_wr_matrix.npy",ng_wr_matrix)
ng_wr_matrix=None

In [ ]:
ng_matrix=np.load("/global/project/projectdirs/des/zhou/sa/data/gammaT/ng_matrix.npy");
ng_wr_matrix=np.load("/global/project/projectdirs/des/zhou/sa/data/gammaT/ng_wr_matrix.npy");

In [ ]:
fig, axs = plt.subplots(3, 4,figsize=(20,15))
for i in range(3):
    for j in range(4):
        values=ng_matrix[i][j]
        values_wr=ng_wr_matrix[i][j]
        axs[i,j].errorbar(np.exp(values[0]),values[1],yerr=np.sqrt(values[2]),label="All sources")
        axs[i,j].errorbar(np.exp(values[0]),values_wr[1],yerr=np.sqrt(values_wr[2]),label="wo/ redMaPPer")
        axs[i,j].set_xscale("log")
        axs[i,j].set_yscale("log")
        axs[i,j].legend()
#         axs.xscale("log")
#         axs[i,j].plot(np.exp(values_wr[0]),values_wr[1])

In [ ]:
fig, axs = plt.subplots(3, 4,figsize=(20,15))
for i in range(3):
    for j in range(4):
        values=ng_matrix[i][j]
        values_wr=ng_wr_matrix[i][j]
        axs[i,j].plot(np.exp(values[0]),values[1]-values_wr[1])
        axs[i,j].set_xscale("log")
        axs[i,j].set_yscale("log")

In [ ]:
np.mean((values[1]-values_wr[1])/values[1])

# Jackknife variance

In [ ]:
## lambda x z = 4x3
import gc
    
ng_matrix=[[None for i in range(4)] for j in range(3)]

for i in trange(3):
    for j in trange(4):
        ng=get_ng_source(clusters,members,z_bins[i],z_bins[i+1],lambda_bins[j],lambda_bins[j+1],foreback="")
        ng_values=np.copy(np.array([ng.meanlogr,ng.xi,ng.varxi]))
        ng_matrix[i][j]=ng_values
        del ng
        gc.collect()

In [ ]:
def get_jk_matrix(z_lower,z_upper,lambda_lower,lambda_upper):
    clusters_z_mask=(clusters[('All','All','Z_LAMBDA')]>=z_lower)&(clusters[('All','All','Z_LAMBDA')]<=z_upper)
    clusters_lambda_mask=(clusters[('All','All','LAMBDA_CHISQ')]>=lambda_lower)&(clusters[('All','All','LAMBDA_CHISQ')]<=lambda_upper) 
    clusters_masked=clusters[(clusters_z_mask)&(cluster_lambda_mask)]
    
    for index,clusters in clusters.iterrows():
        